In [38]:
import torch
import os
from torchvision import datasets, transforms
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.0,), (1,))
])
data="./Fashion Images"
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
trainset = datasets.ImageFolder(root=data+"/train", transform=transform)
testset = datasets.ImageFolder(root=data+"/test", transform=transform)
validset = datasets.ImageFolder(root=data+"/valid", transform=transform)

trainloader = torch.utils.data.DataLoader(dataset=trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(dataset=testset, batch_size=64, shuffle=False)
validloader = torch.utils.data.DataLoader(dataset=validset, batch_size=64, shuffle=False)

In [39]:
print(trainset.class_to_idx)

{'Female': 0, 'Male': 1}


In [40]:
for images, labels in trainloader:
    print(images.shape)
    print(labels)
    break


torch.Size([64, 3, 128, 128])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0])


In [41]:
print(len(trainset),len(testset),len(validset))
print(len(trainloader),len(testloader),len(validloader))

850 50 100
14 1 2


In [42]:
import torch
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(128 * 64 * 64, 256)
        self.fc2 = nn.Linear(256, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.conv3(x)
        x = nn.ReLU()(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

model = CNN()

In [43]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [44]:
checkpoint_dir = 'checkpoints_cnn_fashion_images'
os.makedirs(checkpoint_dir, exist_ok=True)

def find_last_checkpoint():
    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith('model_epoch_')]
    if not checkpoints:
        return None, 0
    checkpoints.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]), reverse=True)
    last_checkpoint = checkpoints[0]
    last_epoch = int(last_checkpoint.split('_')[-1].split('.')[0])
    return os.path.join(checkpoint_dir, last_checkpoint), last_epoch

In [45]:
def evaluate_model(testloader):
    checkpoint_path, start_epoch = find_last_checkpoint()
    if checkpoint_path:
        print(f"Loading latest checkpoint from {checkpoint_path}")
        model.load_state_dict(torch.load(checkpoint_path))
    else:
        print("No checkpoint found. Evaluating with the current model state.")
        return "No checkpoints"
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy}%")
    return accuracy

In [46]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)
def train_model(model, trainloader, criterion, optimizer, num_epochs=10):
    checkpoint_path, start_epoch = find_last_checkpoint()
    if checkpoint_path:
        print(f"Resuming training from {checkpoint_path}")
        model.load_state_dict(torch.load(checkpoint_path))
    else:
        print("No checkpoint found. Starting from scratch.")
        start_epoch = 0
    for epoch in range(start_epoch, num_epochs):
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        checkpoint_path = os.path.join(checkpoint_dir, f'model_epoch_{epoch + 1}.pth')
        torch.save(model.state_dict(), checkpoint_path)
        print(f"Model saved at {checkpoint_path}")
        val_accuracy = evaluate_model(validloader)
        if not isinstance(val_accuracy, float):
            val_accuracy = float(val_accuracy)
        if not isinstance(running_loss, float):
            running_loss = float(running_loss)
        print(f"Epoch {epoch+1}, Validation Accuracy: {val_accuracy:.2f}%, Loss: {running_loss / len(trainloader):.4f}")
train_model(model, trainloader, criterion, optimizer, num_epochs=30)

No checkpoint found. Starting from scratch.
Model saved at checkpoints_cnn_fashion_images/model_epoch_1.pth
Loading latest checkpoint from checkpoints_cnn_fashion_images/model_epoch_1.pth


/var/folders/tq/jtgfkbln5bv4lmlbq2tvnq1h0000gn/T/ipykernel_78849/2950907048.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_p

Accuracy: 77.0%
Epoch 1, Validation Accuracy: 77.00%, Loss: 3.1687
Model saved at checkpoints_cnn_fashion_images/model_epoch_2.pth
Loading latest checkpoint from checkpoints_cnn_fashion_images/model_epoch_2.pth
Accuracy: 83.0%
Epoch 2, Validation Accuracy: 83.00%, Loss: 0.4698
Model saved at checkpoints_cnn_fashion_images/model_epoch_3.pth
Loading latest checkpoint from checkpoints_cnn_fashion_images/model_epoch_3.pth
Accuracy: 85.0%
Epoch 3, Validation Accuracy: 85.00%, Loss: 0.4233
Model saved at checkpoints_cnn_fashion_images/model_epoch_4.pth
Loading latest checkpoint from checkpoints_cnn_fashion_images/model_epoch_4.pth
Accuracy: 80.0%
Epoch 4, Validation Accuracy: 80.00%, Loss: 0.3624
Model saved at checkpoints_cnn_fashion_images/model_epoch_5.pth
Loading latest checkpoint from checkpoints_cnn_fashion_images/model_epoch_5.pth
Accuracy: 79.0%
Epoch 5, Validation Accuracy: 79.00%, Loss: 0.3386
Model saved at checkpoints_cnn_fashion_images/model_epoch_6.pth
Loading latest checkpoint

In [47]:
evaluate_model(testloader)
print(model)

Loading latest checkpoint from checkpoints_cnn_fashion_images/model_epoch_30.pth


/var/folders/tq/jtgfkbln5bv4lmlbq2tvnq1h0000gn/T/ipykernel_78849/2950907048.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_p

Accuracy: 84.0%
CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=524288, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=2, bias=True)
)
